# Tutorial

## Prepare Inputs

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Usage
### Prepare input data
CoMBCR integrates BCRs and gene expressions but requires three files: a BCR sequences file, a gene expression file, and a file containing BCR embeddings generated by a BCR encoder (e.g., AntiBERTa, ESM2).  
- Ensure each file includes an index column labeled "barcode," serving as a unique identifier for each cell.   
- Verify that the cells are aligned in the same order across all three files.
#### BCR sequences file
- This CSV file should include an index column named "barcode" and columns labeled "IGL", and "IGH". 
- The IGL/IHG is the concatnation of "fwr1", "cdr1", "cdr2", "fwr2", "cdr3", "fwr3", and "fwr4" in alpha chain/beta chain.
#### Gene expression file
Normalization and log-transformation are recommended. Batch effect removal is advisable if applicable. We suggest using the top 5,000 highly variable genes, though you can select input genes according to your criteria.


In [2]:
bcr = pd.read_csv("example_pairdata/example_bcr.csv", index_col="barcode")
rna = pd.read_csv("example_pairdata/example_rna.csv", index_col="barcode")
assert(bcr.index.tolist() == rna.index.tolist())

In [3]:
bcr.head()

cdr3L  \
barcode                                   
AAACCTGAGACTTGAA-1_10x_1    CQQYGTAPFTF   
AAACCTGGTAGGGACT-1_10x_1     CQYYGSSPSC   
AAACCTGTCAGTCCCT-1_10x_1   CQLFGDSPMYTF   
AAACGGGAGCGACGTA-1_10x_1  CNSYAGNNNYVLF   
AAACGGGAGGGATGGG-1_10x_1   CYSTDSSGNSLF   

                                                                        IGL  \
barcode                                                                       
AAACCTGAGACTTGAA-1_10x_1  EIVLTQSPGTLSLSPGDRAALSCGASQAVNNNFLAWYQQKPGQAPR...   
AAACCTGGTAGGGACT-1_10x_1  EIVLTQSPGTLSLSPGERATLSCRASQSVSSSLLAWYQQKPGQAPR...   
AAACCTGTCAGTCCCT-1_10x_1  EGVLTQSPGTLSLSPGERATLSCRASQTLNSDFLIWYQLKPGQTPR...   
AAACGGGAGCGACGTA-1_10x_1  QSALTQPPSASGSPGQSVTISCTGTSSDVGDSNYVSWYQQHPGKAP...   
AAACGGGAGGGATGGG-1_10x_1  SYELTQPPSVSVSPGQTARITCSGDALPKKYAYWYQQKSGQAPVLV...   

                                                  cdr3H  \
barcode                                                   
AAACCTGAGACTTGAA-1_10x_1         CARHLKYCTGGSCYSRMVFDSW   
AAACCTGGTAGGGACT-1_10x_1                CASRFGEFLAVCDFW   
AAACCTGTCAGTCCCT-1_10x_1  CAHSRKGFCSGETCYSFLETSGYHWFDPW   
AAACGGGAGCGACGTA-1_10x_1                   CARGVKGRFDYW   
AAACGGGAGGGATGGG-1_10x_1              CARGAQYSSGWYDIDYW   

                                                                        IGH  \
barcode                                                                       
AAACCTGAGACTTGAA-1_10x_1  QVQLQESGPGLVKPSETLSLTCSVSGGSISSFYWSWIRQPPGRGLE...   
AAACCTGGTAGGGACT-1_10x_1  EVQLLESGGGLVQPGGSLRLSCAASGFTFSDHAVSWVRQAAGKGLE...   
AAACCTGTCAGTCCCT-1_10x_1  QVTLKESGPELVKPTQTLTLTCTLSEFPLNSVGMGMGWIRQTPGKT...   
AAACGGGAGCGACGTA-1_10x_1  QLQLQESGSGLVKPSQTLSLTCAVSGGSIGSSSYSWSWIRQPPGKG...   
AAACGGGAGGGATGGG-1_10x_1  QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYGISWVRQAPGQGLE...   

                         raw_clonotype_id sample  
barcode                                           
AAACCTGAGACTTGAA-1_10x_1    clonotype2764  10x_1  
AAACCTGGTAGGGACT-1_10x_1    clonotype1232  10x_1  
AAACCTGTCAGTCCCT-1_10x_1     clonotype211  10x_1  
AAACGGGAGCGACGTA-1_10x_1      clonotype88  10x_1  
AAACGGGAGGGATGGG-1_10x_1    clonotype1765  10x_1

In [4]:
rna.head()

IGHV3.15  IGHV3.48  IGHV1.18  IGHV4.34  IGHV5.51  \
barcode                                                                      
AAACCTGAGACTTGAA-1_10x_1 -0.376008 -0.065586  0.914290  0.021602 -0.099588   
AAACCTGGTAGGGACT-1_10x_1  0.658475  0.784789 -0.049671 -0.285894  0.017293   
AAACCTGTCAGTCCCT-1_10x_1 -0.211308  0.152276  0.209336  0.028572 -0.246877   
AAACGGGAGCGACGTA-1_10x_1  0.042634  0.478668  0.106011  0.324261  0.064261   
AAACGGGAGGGATGGG-1_10x_1  0.045003  0.126678  3.847370 -0.160274 -0.264143   

                           IGHV2.5  IGHV3.53   IGLV2.8   IGHV1.2  IGHV1.69D  \
barcode                                                                       
AAACCTGAGACTTGAA-1_10x_1 -0.136340 -0.234204 -0.149438  0.105756   0.006067   
AAACCTGGTAGGGACT-1_10x_1  0.118015  0.530007 -0.092610  0.086478   0.224125   
AAACCTGTCAGTCCCT-1_10x_1  5.379848  0.053748 -0.031826  0.106044   0.667313   
AAACGGGAGCGACGTA-1_10x_1 -0.292733  0.874743  6.847327  0.020781   0.681784   
AAACGGGAGGGATGGG-1_10x_1  0.090358 -0.008701  0.184167  0.298701  -0.029724   

                          ...  RAD21.AS1    UBE2L5  AC005962.2   FAM167A  \
barcode                   ...                                              
AAACCTGAGACTTGAA-1_10x_1  ...   0.000000  0.007155    0.000000  0.000000   
AAACCTGGTAGGGACT-1_10x_1  ...   0.000000  0.003563    0.000000  0.000918   
AAACCTGTCAGTCCCT-1_10x_1  ...   0.000000  0.000000   -0.039742  0.000000   
AAACGGGAGCGACGTA-1_10x_1  ...  -0.007744  0.065336    0.000000  0.000000   
AAACGGGAGGGATGGG-1_10x_1  ...   0.000000  0.000000    0.000000  0.000000   

                          AC092428.1   CEACAM1  AC114980.1  AC019294.2  \
barcode                                                                  
AAACCTGAGACTTGAA-1_10x_1   -0.041768  0.000000    0.000000    0.000000   
AAACCTGGTAGGGACT-1_10x_1    0.000000  0.000000    0.000000    0.023573   
AAACCTGTCAGTCCCT-1_10x_1    0.000000  0.005113    0.000000    0.000000   
AAACGGGAGCGACGTA-1_10x_1   -0.001011  0.000000    0.002812   -0.009169   
AAACGGGAGGGATGGG-1_10x_1    0.000000  0.000000    0.000000    0.000000   

                             HSPA2  DENND5B.AS1  
barcode                                          
AAACCTGAGACTTGAA-1_10x_1 -0.001868    -0.009476  
AAACCTGGTAGGGACT-1_10x_1  0.000000     0.003488  
AAACCTGTCAGTCCCT-1_10x_1  0.000000     0.001509  
AAACGGGAGCGACGTA-1_10x_1  0.000000     0.008073  
AAACGGGAGGGATGGG-1_10x_1  0.000000    -0.010457  

[5 rows x 5000 columns]

### Genreate original BCR embeddings

First, we download the pre-trained BCR encoder


In [5]:
from CoMBCR.utils import download_BCRencoder

download_BCRencoder()

Fetching 8 files: 100%|██████████| 8/8 [00:28<00:00,  3.57s/it]

Download Finished. Path /mnt/d/CoMBCR/CoMBCR/BCRencoder


Please clone or download the "runberta_pair.py" in this github. This file is used to measure the original distances between BCRs. We recommend using our default pre-trained encoder, though any encoder can be used to encode BCRs.

```
python3 runberta_pair.py --datapath "example_pairdata/example_bcr.csv" --outdir "example_outdir" --outfilename "antiberta_embedding.csv"
```

Here we directly used the original BCR embeddings under the exampledata

In [5]:
bcrori = pd.read_csv("example_pairdata/example_bcrori.csv", index_col="barcode")
bcrori.head()

0         1         2         3         4  \
barcode                                                                      
AAACCTGAGACTTGAA-1_10x_1 -0.026743 -0.664474 -0.444477  0.982374 -0.056200   
AAACCTGGTAGGGACT-1_10x_1 -0.514521 -0.598449 -0.404644  1.021118 -0.253018   
AAACCTGTCAGTCCCT-1_10x_1  0.082168 -0.437680 -0.455506  1.005682 -0.099462   
AAACGGGAGCGACGTA-1_10x_1  0.137382 -0.862890 -0.425157  0.813950 -0.498311   
AAACGGGAGGGATGGG-1_10x_1  0.161128 -0.334227 -0.600657  0.650591 -0.759796   

                                 5         6         7         8         9  \
barcode                                                                      
AAACCTGAGACTTGAA-1_10x_1 -0.090404 -0.181163  0.064309  0.139462 -0.009462   
AAACCTGGTAGGGACT-1_10x_1  0.093992 -0.366508  0.177612 -0.104526 -0.050579   
AAACCTGTCAGTCCCT-1_10x_1  0.034462  0.027507 -0.016120 -0.080524 -0.378358   
AAACGGGAGCGACGTA-1_10x_1  0.072822  0.123834  0.367978 -0.162537 -0.148761   
AAACGGGAGGGATGGG-1_10x_1  0.213413  0.108718  0.077125  0.016716 -0.003692   

                          ...      1014      1015      1016      1017  \
barcode                   ...                                           
AAACCTGAGACTTGAA-1_10x_1  ... -0.152017  0.215193  0.473038 -0.206209   
AAACCTGGTAGGGACT-1_10x_1  ...  0.010027  0.321669  0.113003 -0.126156   
AAACCTGTCAGTCCCT-1_10x_1  ...  0.431250  0.392155  0.551969 -0.131442   
AAACGGGAGCGACGTA-1_10x_1  ...  0.313735  0.489395  0.266964 -0.418712   
AAACGGGAGGGATGGG-1_10x_1  ...  0.614696  0.969117  0.398579 -0.450386   

                              1018      1019      1020      1021      1022  \
barcode                                                                      
AAACCTGAGACTTGAA-1_10x_1  0.109365 -0.108639 -0.367183  0.309040 -0.357757   
AAACCTGGTAGGGACT-1_10x_1  0.341789 -0.211246 -0.517015  0.235695 -0.102193   
AAACCTGTCAGTCCCT-1_10x_1 -0.194720 -0.522121 -0.149101  0.197037 -0.391534   
AAACGGGAGCGACGTA-1_10x_1  0.241340 -0.303050 -0.243020  0.138295 -0.184176   
AAACGGGAGGGATGGG-1_10x_1  0.144531 -0.781121 -0.327100 -0.423524 -0.357415   

                              1023  
barcode                             
AAACCTGAGACTTGAA-1_10x_1 -0.589497  
AAACCTGGTAGGGACT-1_10x_1 -0.722848  
AAACCTGTCAGTCCCT-1_10x_1 -0.281698  
AAACGGGAGCGACGTA-1_10x_1 -0.638734  
AAACGGGAGGGATGGG-1_10x_1  0.369593  

[5 rows x 1024 columns]

In [6]:
assert(bcr.index.tolist() == bcrori.index.tolist())

### Run CoMBCR

## steps
CoMBCR contains the parameters as follows:
1. bcrpath: (Required) The path to the BCR sequences file.
2. rnapath:  (Required) The path to the gene expression file.
3. bcroriginal: (Required) The path to bcr original embedding file.
4. outdir:  (Required) The directory where the best checkpoint file and the output embeddings will be stored.
5. checkpoint: default is "best_network.pth". This parameter specifies the name of the file where the best model checkpoint will be saved.
6. lr: default is 1e-6.
7. lam: default is 1e-1, the inner parameter (Parameter alpha in paper).
8. batch_size: default 256.
9. epochs: default 200.
10. patience: default 15, the patience for early stopping.
11. lr_step: default [40,100], These are the milestones for the MultiStepLR setting, which adjusts the learning rate at specified epochs.
12. encoderprofile_in_dim: default 5000. Adjust this parameter if the number of input genes differs from 5000.
13. separatebatch: The default is False. If set to True, BCRs from different samples will be treated as distinct BCRs. Ensure that your BCR input file contains a "sample" column if you choose to enable this option.
14. IGH_clonotype: The default is False. If set to True, BCRs with identical heavy chains will be regarded as a clonotype. 

The codes below return numpy arrays for BCR embeddings and gex embeddings. Meanwhile, it will output "bcrembedding.csv" and "gexemedding.csv" under the outdir you designated.  

If CUDA raised error, this is due to a crush with the previous loaded model. Please restart the jupyter notebook and directly run the cell below.

In [2]:
from CoMBCR.CoMBCRpair import CoMBCR_main

bcremb, gexemb = CoMBCR_main(bcrpath="example_pairdata/example_bcr.csv", 
            rnapath="example_pairdata/example_rna.csv", 
            bcroriginal="example_pairdata/example_bcrori.csv", 
            outdir="example_outdir",
            epochs=1,
            batch_size=16,
            encoderprofile_in_dim=5000)

learning rate is  1e-06
Adjusting learning rate of group 0 to 1.0000e-06.
Adjusting learning rate of group 0 to 1.0000e-06.
Epoch:[0/1]	loss:5.74803	loss_cmc:2.776516	loss_p2p:2.987942	loss_b2b:0.000905


In [11]:
bcremb.shape

(1000, 256)

In [12]:
gexemb.shape

(1000, 256)

Read the output files "bcrembedding.csv" and "gexembedding.csv" located in the designated output directory. Please note that these CSV files directly store the numpy arrays and, as such, do not include any "barcode" column. When reading these files, ensure that you do not specify any index column.

In [ ]:
bcremb = pd.read_csv("example_outdir/Embeddings/bcrembeddings.csv")
gexemb = pd.read_csv("example_outdir/Embeddings/gexembeddings.csv")

In [14]:
bcremb.head()

0         1         2         3         4         5         6  \
0  0.015348 -0.087860 -0.013923  0.056350 -0.017100  0.068572  0.005855   
1  0.124641  0.031098  0.027163  0.138613 -0.001733  0.038861  0.051714   
2  0.035745 -0.049108 -0.037657  0.123449  0.000613  0.055640  0.086152   
3  0.087479  0.025641 -0.032866  0.033859  0.027408 -0.012790  0.118991   
4  0.013984 -0.057683 -0.067793  0.080526 -0.009838  0.049828  0.068106   

          7         8         9  ...       246       247       248       249  \
0 -0.050495 -0.005957  0.008949  ... -0.136085 -0.076334 -0.083860 -0.018186   
1  0.011094 -0.049876  0.009269  ... -0.106010  0.002925 -0.024259 -0.017597   
2  0.093559  0.029612 -0.020621  ... -0.182125  0.023510 -0.041843 -0.049026   
3  0.037054 -0.002722 -0.013472  ... -0.134776  0.024128  0.035573 -0.098892   
4  0.017483  0.001821  0.027285  ... -0.127777 -0.004428 -0.013432 -0.056396   

        250       251       252       253       254       255  
0 -0.109076 -0.055655  0.039805 -0.069356 -0.029027 -0.031129  
1 -0.042338 -0.005912  0.011963 -0.059456  0.017165  0.023257  
2 -0.042913  0.086104 -0.060631 -0.022182  0.026433  0.007512  
3  0.000892  0.075088  0.008346 -0.024887 -0.018854  0.046138  
4 -0.009095  0.052927 -0.116035  0.022259 -0.001118 -0.014250  

[5 rows x 256 columns]

In [15]:
gexemb.head()

0         1         2         3         4         5         6  \
0 -0.048509 -0.050780  0.012092  0.054801 -0.077096 -0.071795  0.120492   
1  0.008329 -0.065424  0.061591 -0.017872 -0.059715 -0.040780  0.160052   
2 -0.035611 -0.070885  0.032202  0.049993 -0.043414 -0.038092  0.121762   
3 -0.063172 -0.066526  0.050900  0.066959 -0.041829 -0.060583  0.122334   
4 -0.012943 -0.077301 -0.010864  0.059000 -0.085983 -0.026742  0.133180   

          7         8         9  ...       246       247       248       249  \
0  0.048243 -0.091755 -0.037499  ...  0.030512 -0.069922 -0.017588 -0.057541   
1 -0.033229 -0.061964 -0.026953  ... -0.013495 -0.101155  0.022401 -0.036999   
2  0.008126 -0.090482 -0.002106  ...  0.000275 -0.075901  0.017932 -0.014056   
3 -0.033201 -0.063927  0.005277  ... -0.014794 -0.117377  0.009713 -0.037630   
4  0.004001 -0.066243 -0.045583  ... -0.045505 -0.119614  0.013246 -0.047752   

        250       251       252       253       254       255  
0 -0.033826 -0.059658 -0.007020 -0.018299  0.034806  0.077380  
1 -0.039105 -0.100446 -0.006467 -0.010011  0.036884  0.089870  
2 -0.033765 -0.051641 -0.003662 -0.027082  0.036087  0.060277  
3 -0.051727 -0.059814 -0.001520 -0.034314  0.050281  0.058940  
4 -0.039822 -0.060070  0.023413  0.028433  0.026463  0.055106  

[5 rows x 256 columns]